In [1]:
import pyspark
import sys 
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import col 
from pyspark.sql.functions import expr




In [3]:
spark = SparkSession.builder.getOrCreate()
# df = spark.read.text("wordcount.txt")
df = spark.read.option("recursiveFileLookup", "true").csv("/home/lplab/Desktop/210962018/lab3/donation/block_*.csv")
df.show()


+-----+-----+-----------------+------------+------------+------------+-------+------+------+------+-------+--------+
|  _c0|  _c1|              _c2|         _c3|         _c4|         _c5|    _c6|   _c7|   _c8|   _c9|   _c10|    _c11|
+-----+-----+-----------------+------------+------------+------------+-------+------+------+------+-------+--------+
| id_1| id_2|     cmp_fname_c1|cmp_fname_c2|cmp_lname_c1|cmp_lname_c2|cmp_sex|cmp_bd|cmp_bm|cmp_by|cmp_plz|is_match|
|37291|53113|0.833333333333333|           ?|           1|           ?|      1|     1|     1|     1|      0|    TRUE|
|39086|47614|                1|           ?|           1|           ?|      1|     1|     1|     1|      1|    TRUE|
|70031|70237|                1|           ?|           1|           ?|      1|     1|     1|     1|      1|    TRUE|
|84795|97439|                1|           ?|           1|           ?|      1|     1|     1|     1|      1|    TRUE|
|36950|42116|                1|           ?|           1|       

In [5]:
# CLEANING AND PREPROCESS DATA


spark = SparkSession.builder.getOrCreate()

df = spark.read.option("header","true").option("nullValue","?").option("inferSchema","true").csv("donation/block_1.csv")

df.show()

print("Count of Dataframe with all 10 .csv files loaded = ",df.count())


+-----+-----+-----------------+------------+------------+------------+-------+------+------+------+-------+--------+
| id_1| id_2|     cmp_fname_c1|cmp_fname_c2|cmp_lname_c1|cmp_lname_c2|cmp_sex|cmp_bd|cmp_bm|cmp_by|cmp_plz|is_match|
+-----+-----+-----------------+------------+------------+------------+-------+------+------+------+-------+--------+
|37291|53113|0.833333333333333|        null|         1.0|        null|      1|     1|     1|     1|      0|    true|
|39086|47614|              1.0|        null|         1.0|        null|      1|     1|     1|     1|      1|    true|
|70031|70237|              1.0|        null|         1.0|        null|      1|     1|     1|     1|      1|    true|
|84795|97439|              1.0|        null|         1.0|        null|      1|     1|     1|     1|      1|    true|
|36950|42116|              1.0|        null|         1.0|         1.0|      1|     1|     1|     1|      1|    true|
|42413|48491|              1.0|        null|         1.0|       

In [6]:
df.printSchema()

df1 = df[df['is_match'] == 'false']
df1.show()


root
 |-- id_1: integer (nullable = true)
 |-- id_2: integer (nullable = true)
 |-- cmp_fname_c1: double (nullable = true)
 |-- cmp_fname_c2: double (nullable = true)
 |-- cmp_lname_c1: double (nullable = true)
 |-- cmp_lname_c2: double (nullable = true)
 |-- cmp_sex: integer (nullable = true)
 |-- cmp_bd: integer (nullable = true)
 |-- cmp_bm: integer (nullable = true)
 |-- cmp_by: integer (nullable = true)
 |-- cmp_plz: integer (nullable = true)
 |-- is_match: boolean (nullable = true)

+-----+-----+-----------------+------------+-----------------+------------+-------+------+------+------+-------+--------+
| id_1| id_2|     cmp_fname_c1|cmp_fname_c2|     cmp_lname_c1|cmp_lname_c2|cmp_sex|cmp_bd|cmp_bm|cmp_by|cmp_plz|is_match|
+-----+-----+-----------------+------------+-----------------+------------+-------+------+------+------+-------+--------+
|20206|55692|              1.0|        null|0.166666666666667|        null|      1|     0|     0|     1|      0|   false|
|13466|69531|     

In [7]:
df.groupBy("is_match").count().orderBy(col("count").desc()).show()

summary = df.describe()
summary.select("summary", "cmp_fname_c1", "cmp_fname_c2").show()

matches = df.where("is_match = true")
match_summary = matches.describe()
match_summary.select("summary","cmp_fname_c1","cmp_fname_c2").show()

misses = df.where("is_match = false")
misses_summary = misses.describe()
misses_summary.select("summary","cmp_fname_c1","cmp_fname_c2").show()

+--------+------+
|is_match| count|
+--------+------+
|   false|572820|
|    true|  2093|
+--------+------+

+-------+------------------+------------------+
|summary|      cmp_fname_c1|      cmp_fname_c2|
+-------+------------------+------------------+
|  count|            574811|             10325|
|   mean|0.7127592938253411|0.8977586763518969|
| stddev|0.3889286452463531|0.2742577520430532|
|    min|               0.0|               0.0|
|    max|               1.0|               1.0|
+-------+------------------+------------------+

+-------+-------------------+-------------------+
|summary|       cmp_fname_c1|       cmp_fname_c2|
+-------+-------------------+-------------------+
|  count|               2091|                128|
|   mean| 0.9970329792424486| 0.9955357142857143|
| stddev|0.03979189523588238|0.05050762722761048|
|    min|                0.0|  0.428571428571429|
|    max|                1.0|                1.0|
+-------+-------------------+-------------------+

+------

# question 2 or similarity score

In [8]:
good_features = ["cmp_lname_c1","cmp_plz","cmp_by","cmp_bd","cmp_bm"]

sum_expression = " + ".join(good_features)

scored = df.fillna(0,subset = good_features).withColumn('score',expr(sum_expression)).select('score','is_match')

scored.show()

+-----+--------+
|score|is_match|
+-----+--------+
|  4.0|    true|
|  5.0|    true|
|  5.0|    true|
|  5.0|    true|
|  5.0|    true|
|  5.0|    true|
|  5.0|    true|
|  4.0|    true|
|  5.0|    true|
|  5.0|    true|
|  5.0|    true|
|  5.0|    true|
|  5.0|    true|
|  5.0|    true|
|  4.0|    true|
|  5.0|    true|
|  5.0|    true|
|  5.0|    true|
|  5.0|    true|
|  5.0|    true|
+-----+--------+
only showing top 20 rows



question 3 or Calculating Precision,Recall and F1-Score

In [9]:
def calculate_metrics(scored, threshold):
    tp = scored.filter((col('score') >= threshold) & (col('is_match') == 'true')).count()
    fp = scored.filter((col('score') >= threshold) & (col('is_match') == 'false')).count()
    fn = scored.filter((col('score') < threshold) & (col('is_match') == 'true')).count()

    precision = tp / (tp + fp) if (tp + fp) != 0 else 0
    recall = tp / (tp + fn) if (tp + fn) != 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

    return precision, recall, f1_score

threshold = 4.0

precision, recall, f1_score = calculate_metrics(scored, threshold)

print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1_score}")

Precision: 0.9693450998606595
Recall: 0.9971333014811276
F1 Score: 0.9830428638718794
